In [1]:
import pandas as pd
import numpy as np
import json
import os
import re
from tqdm.notebook import tqdm
tqdm.pandas()
import sklearn.feature_extraction
import itertools, string, operator, re, unicodedata, nltk
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer
import nltk
from nltk.corpus import stopwords
'''Features'''
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import TruncatedSVD
#from sklearn.preprocessing import label_binarize
'''Classifiers'''
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
'''Metrics/Evaluation'''
from sklearn.model_selection import train_test_split
#from sklearn.model_selection import GridSearchCV
#from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_curve, auc, confusion_matrix
#from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfTransformer
#from sklearn.naive_bayes import MultinomialNB
#from sklearn.ensemble import RandomForestClassifier
import warnings
warnings.simplefilter(action='ignore')
import nltk
nltk.download('averaged_perceptron_tagger')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/jghosh2/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/jghosh2/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/jghosh2/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /Users/jghosh2/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [2]:
df = pd.read_csv('Taxml_Integers_Verified.csv', index_col=None, header=0,dtype={'item_name': str, 'description': str,'establishment_type': str, 'CAT_Name':str},usecols=['item_name','description','establishment_type','CAT_Name'])

In [3]:
le=WordNetLemmatizer()
def get_wordnet_pos(word):
    """Map POS tag to first character lemmatize() accepts"""
    tag = nltk.pos_tag([word])[0][1][0].upper()
    #print(tag,word)
    tag_dict = {"J": wordnet.ADJ,
                "N": wordnet.NOUN,
                "V": wordnet.VERB,
                "R": wordnet.ADV}
    return tag_dict.get(tag, wordnet.NOUN)
def clean_text(text):

    text = text.lower() # lowercase text
    text = REPLACE_BY_SPACE_RE.sub(' ', text) # replace REPLACE_BY_SPACE_RE symbols by space in text. substitute the matched string in REPLACE_BY_SPACE_RE with space.
    text = BAD_SYMBOLS_RE.sub('', text) # remove symbols which are in BAD_SYMBOLS_RE from text. substitute the matched string in BAD_SYMBOLS_RE with nothing. 
    text = ' '.join(word for word in list(set(text.split())) if word not in STOPWORDS)# remove stopwors from text
    #text =  " ".join([le.lemmatize(w, get_wordnet_pos(w)) for w in nltk.word_tokenize(text) if w not in string.punctuation])
    text =  " ".join([le.lemmatize(w) for w in nltk.word_tokenize(text) if w not in string.punctuation])
    return text

def cat_name_sorted(text):
    text=sorted(text.split(','))
    text=','.join([x for x in text])
    return text
def cat_name_without_temp(text):
    if ',TEMP_HEATED' in text:
        text=re.sub(',TEMP_HEATED','', text)
    if ',TEMP_UNHEATED' in text:
        text=re.sub(',TEMP_UNHEATED','', text)
    if ',TEMP_COLD' in text:
        text=re.sub(',TEMP_COLD','', text)
    if 'TEMP_HEATED' in text:
        text=re.sub('TEMP_HEATED','', text)
    if 'TEMP_UNHEATED' in text:
        text=re.sub('TEMP_UNHEATED','', text)
    if 'TEMP_COLD' in text:
        text=re.sub('TEMP_COLD','', text)
    else:
        pass
    return text



In [4]:
df.shape

(175570, 4)

In [5]:
REPLACE_BY_SPACE_RE = re.compile('[/(){}\[\]\|@,;]')
BAD_SYMBOLS_RE = re.compile('[^0-9a-z #+_]')
STOPWORDS = list(set(stopwords.words('english')))
STOPWORDS.append('n')
STOPWORDS.append('grocery')
#df1=df.sample(n=2000,random_state=42)
df1=df.sample(frac=1, random_state=42)
df1['CAT_Name']=df1['CAT_Name'].progress_apply(lambda x: cat_name_sorted(x))
df1['CAT_Name']=df1['CAT_Name'].progress_apply(lambda x: cat_name_without_temp(x))
df1=df1.replace(r'^\s*$', np.nan, regex=True)
df1 = df1.dropna(subset=['CAT_Name'])
df1=df1.drop_duplicates(subset=['item_name','description','establishment_type','CAT_Name'],ignore_index=True,keep=False)
df1=df1.drop_duplicates(subset=['item_name','description','establishment_type'],ignore_index=True,keep=False)
df1['input_str'] = df1[['item_name', 'description','establishment_type']].apply(lambda x: ' '.join(x[x.notnull()]), axis = 1)
df1['input_str'] = df1['input_str'].apply(clean_text)
df1['input_str'] = df1['input_str'].str.replace('\d+', '')
df1=df1.drop_duplicates(subset=['CAT_Name','input_str'],ignore_index=True,keep='first')
df1 = df1.reset_index(drop=True)

  0%|          | 0/175570 [00:00<?, ?it/s]

  0%|          | 0/175570 [00:00<?, ?it/s]

In [6]:
df1.shape

(151456, 5)

In [7]:
df1

,item_name,description,establishment_type,CAT_Name,input_str
0,Iles Formula Spa Pack,NaN,GROCERY,CAT_TPP,iles formula spa pack
1,Stoli® Gluten Free Vodka.750ml Bottle,NaN,GROCERY,CAT_LIQUOR,gluten stoli bottle vodkaml free
2,"Russian Standard Gold (1L), (1L) Wine & Bubbly...",NaN,GROCERY,CAT_SPARKLING_WINE,standard bubbly l russian abv gold wine
3,"Bell's Two Hearted Ale IPA, 12pk-12oz can beer...",NaN,GROCERY,CAT_BEER,beer bell pkoz ale abv two hearted ipa
4,Barbancourt 8yr Rhum,\N,GROCERY,CAT_LIQUOR,yr barbancourt rhum
...,...,...,...,...,...
151451,"Donjulio 1942 Anejo, 750 ml tequila (40% ABV)",\N,GROCERY,CAT_LIQUOR,ml tequila anejo abv donjulio
151452,Mishka Beef Cake Bubble,Our classic Mishka cakes fit in the palm of yo...,PET,CAT_PET_FOOD,bubble mishka shape signature hand palm cake c...
151453,Oberto Beef Jerky,3.25 oz Original,GROCERY,CAT_PREPARED_FOOD,oberto jerky original beef oz
151454,Rastafri Malibu Afro Kinky Braid Hair - 14 in,\N,GROCERY,CAT_TPP,braid rastafri hair kinky malibu afro


In [151]:
df1.to_csv('final_df11.csv')

In [8]:
df1['cat_count'] = df1.groupby('CAT_Name')['CAT_Name'].transform('count')

In [9]:
df2=df1[df1['cat_count']==1].reset_index()

In [10]:
df3=df1[df1['cat_count']>1].reset_index()

In [11]:
#Train test split with stratified sampling for evaluation
X_train, X_test, y_train, y_test = train_test_split(df3['input_str'],
                                                    df3['CAT_Name'],
                                                    test_size = .20, 
                                                    random_state = 42,
                                                    stratify=df3['CAT_Name'])

In [12]:
y_train=y_train

In [13]:
X_train_final=pd.concat([X_train, df2['input_str']])
y_train_final=pd.concat([y_train, df2['CAT_Name']])
X_train_final=X_train_final.values
y_train_final=y_train_final.values

In [14]:
#from sklearn.naive_bayes import MultinomialNB


xgb = Pipeline([('vect', CountVectorizer()),
               ('tfidf', TfidfTransformer()),
               ('clf', XGBClassifier()),
              ])
xgb.fit(X_train_final, y_train_final)

y_pred = xgb.predict(X_test)

print(f'accuracy {accuracy_score(y_pred,y_test)}')
print(precision_score(y_pred,y_test,average='macro'))
print(recall_score(y_pred,y_test,average='macro'))
print(f1_score(y_pred,y_test,average='macro'))

[15:46:37] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
accuracy 0.7086827335754374
0.5305261851321219
0.5954867163176474
0.5486279692306606


0.6046686002226973
0.704864008931533
0.6341608455538031


In [133]:
result=pd.DataFrame()
result['input_str']=X_test
result['original_cat']=y_test
result['pred']=y_pred
misclassifications = result.loc[result['original_cat']!=result['pred']]
print(misclassifications.shape)
print(result.shape)
misclassifications.to_csv('misclassifications.csv')

(5533, 3)
(30165, 3)


In [135]:
import pickle
# save the model to disk
filename = 'finalized_model.sav'
pickle.dump(rf, open(filename, 'wb'))
 
# load the model from disk
loaded_model = pickle.load(open(filename, 'rb'))
result = loaded_model.score(X_test, y_test)


In [150]:
print(loaded_model.score(x_val, y_val))

0.8996353389690038


In [149]:
#Train test split with stratified sampling for evaluation
X_train, x_val, y_train, y_val = train_test_split(df3['input_str'],
                                                    df3['CAT_Name'],
                                                    test_size = .4, 
                                                    random_state = 42,
                                                    stratify=df3['CAT_Name'])

In [123]:

model_dict = {'Random Forest': Pipeline([('vect', CountVectorizer()),
               ('tfidf', TfidfTransformer()),
               ('rf', RandomForestClassifier()),
              ]),
             
             'naive bayas': Pipeline([('vect', CountVectorizer()),
               ('tfidf', TfidfTransformer()),
               ('clf', MultinomialNB()),
              ]),
              'logistic': Pipeline([('vect', CountVectorizer()),
               ('tfidf', TfidfTransformer()),
               ('lg', LogisticRegression()),
              ])
             }
            

#Function to get the scores for each model in a df
def model_score_df(model_dict):   
    model_name, ac_score_list, p_score_list, r_score_list, f1_score_list = [], [], [], [], []
    for k,v in tqdm(model_dict.items()):   
        model_name.append(k)
        v.fit(X_train_final, y_train_final)
        y_pred = v.predict(X_test)
        ac_score_list.append(accuracy_score(y_test, y_pred))
        p_score_list.append(precision_score(y_test, y_pred, average='macro'))
        r_score_list.append(recall_score(y_test, y_pred, average='macro'))
        f1_score_list.append(f1_score(y_test, y_pred, average='macro'))
        model_comparison_df = pd.DataFrame([model_name, ac_score_list, p_score_list, r_score_list, f1_score_list]).T
        model_comparison_df.columns = ['model_name', 'accuracy_score', 'precision_score', 'recall_score', 'f1_score']
        model_comparison_df = model_comparison_df.sort_values(by='f1_score', ascending=False)
    return model_comparison_df


In [124]:
print(model_score_df(model_dict))

  0%|          | 0/3 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [ ]:
X_train

In [ ]:
#Creating the features (tf-idf weights) for the processed text

#texts = df1['input_str'].astype('str')

tfidf_vectorizer = TfidfVectorizer(ngram_range=(1, 2),max_df = .95)
LE = LabelEncoder()
#tfidf
tfv = TfidfVectorizer(strip_accents='unicode', analyzer='word',token_pattern=r'\w{1,}',ngram_range=(1,2), use_idf=1,smooth_idf=1,sublinear_tf=1,max_df = .95,stop_words = 'english')

X_train_tfidf = tfidf_vectorizer.fit_transform(X_train) #features
X_test_tfidf= tfidf_vectorizer.fit_transform(X_test) #features

tfv.fit(list(X_train) + list(X_test))
xtrain_tfv =  tfv.transform(X_train) 
xvalid_tfv = tfv.transform(X_test)

#countvec
ctv = CountVectorizer(analyzer='word',token_pattern=r'\w{1,}',
            ngram_range=(1, 3), stop_words = 'english')
X_train_ctv = ctv.fit_transform(X_train) #features
X_test_ctv= ctv.fit_transform(X_test) #features



y_train_final=LE.fit_transform(y_train)
y_test_final=LE.fit_transform(y_test)

In [ ]:

#Dimenionality reduction. Only using the 100 best features er category

lsa = TruncatedSVD(n_components=100, 
                   n_iter=10, 
                   random_state=3)

X_train_final = lsa.fit_transform(X_train_tfidf)
X_test_final = lsa.fit_transform(X_test_tfidf)

X_train_final_ctv= lsa.fit_transform(X_train_ctv)
X_test_final_ctv= lsa.fit_transform(X_test_ctv)

In [ ]:
y_pred

In [ ]:
model_name, ac_score_list, p_score_list, r_score_list, f1_score_list = [], [], [], [], []
randomforestmodel=RandomForestClassifier(random_state=3)
#tfidf
#randomforestmodel.fit(xtrain_tfv, y_train_final)
#y_pred = randomforestmodel.predict(xvalid_tfv)
#countvec
#randomforestmodel.fit(X_train_final_ctv, y_train_final)
#y_pred = randomforestmodel.predict(X_test_final_ctv)
#svd
randomforestmodel.fit(X_train_final, y_train_final)
y_pred = randomforestmodel.predict(X_test_final)

ac_score_list.append(accuracy_score(y_test_final, y_pred))
p_score_list.append(precision_score(y_test_final, y_pred, average='macro'))
r_score_list.append(recall_score(y_test_final, y_pred, average='macro'))
f1_score_list.append(f1_score(y_test_final, y_pred, average='macro'))
model_comparison_df = pd.DataFrame([model_name, ac_score_list, p_score_list, r_score_list, f1_score_list]).T
model_comparison_df.columns = ['model_name', 'accuracy_score', 'precision_score', 'recall_score', 'f1_score']
model_comparison_df = model_comparison_df.sort_values(by='f1_score', ascending=False)
print(model_comparison_df)

In [ ]:

#Preliminary model evaluation using default parameters
from sklearn.naive_bayes import MultinomialNB
#Creating a dict of the models
model_dict = {'Random Forest': RandomForestClassifier(random_state=3)}
            

#Function to get the scores for each model in a df
def model_score_df(model_dict):   
    model_name, ac_score_list, p_score_list, r_score_list, f1_score_list = [], [], [], [], []
    for k,v in tqdm(model_dict.items()):   
        model_name.append(k)
        v.fit(X_train_final, y_train_final)
        y_pred = v.predict(X_test_final)
        ac_score_list.append(accuracy_score(y_test_final, y_pred))
        p_score_list.append(precision_score(y_test_final, y_pred, average='macro'))
        r_score_list.append(recall_score(y_test_final, y_pred, average='macro'))
        f1_score_list.append(f1_score(y_test_final, y_pred, average='macro'))
        model_comparison_df = pd.DataFrame([model_name, ac_score_list, p_score_list, r_score_list, f1_score_list]).T
        model_comparison_df.columns = ['model_name', 'accuracy_score', 'precision_score', 'recall_score', 'f1_score']
        model_comparison_df = model_comparison_df.sort_values(by='f1_score', ascending=False)
    return model_comparison_df,v



In [ ]:
final_dict=model_score_df(model_dict)

In [ ]:
final_dict[1]

In [ ]:
print(final_dict[0])

In [45]:
from sklearn.model_selection import RandomizedSearchCV

In [105]:
#clf = RandomForestClassifier()
from pprint import pprint
from time import time
import logging
rf=Pipeline([('vect', CountVectorizer()),
               ('tfidf', TfidfTransformer()),
               ('clf', RandomForestClassifier()),
              ])

parameters = {
    "vect__max_df": (0.5, 0.75, 1.0),
    # 'vect__max_features': (None, 5000, 10000, 50000),
    "vect__ngram_range": ((1, 1), (1, 2)),  # unigrams or bigramslf__
     "clf__bootstrap":[True, False],
     "clf__max_depth":[10, 50, 100,500, None],
     "clf__max_features":['auto', 'sqrt'],
     "clf__min_samples_leaf":[1,2,4],
     "clf__min_samples_split":[2,5,10],
     "clf__n_estimators":[400,600,800],
     "clf__random_state":[3]

}

RandomizedSearch = RandomizedSearchCV(rf,
                          parameters, 
                          cv=5,
                          verbose=1, 
                          n_jobs=-1)

t0 = time()
rf_best_model = RandomizedSearch.fit(X_train_final, y_train_final)
print("done in %0.3fs" % (time() - t0))
#print()
print("Best score: %0.3f" % rf_best_model.best_score_)
print("Best parameters set:")
best_parameters = rf_best_model.best_estimator_.get_params()
for param_name in sorted(parameters.keys()):
    print("\t%s: %r" % (param_name, best_parameters[param_name]))


Fitting 5 folds for each of 10 candidates, totalling 50 fits


/opt/anaconda3/lib/python3.8/site-packages/sklearn/model_selection/_split.py:676: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(


done in 27.282s
Best score: 0.592
Best parameters set:
	clf__bootstrap: True
	clf__max_depth: 100
	clf__max_features: 'sqrt'
	clf__min_samples_leaf: 1
	clf__min_samples_split: 2
	clf__n_estimators: 400
	clf__random_state: 3
	vect__max_df: 1.0
	vect__ngram_range: (1, 1)


In [103]:
rf_best_model

RandomizedSearchCV(cv=5,
                   estimator=Pipeline(steps=[('vect', CountVectorizer()),
                                             ('tfidf', TfidfTransformer()),
                                             ('clf',
                                              RandomForestClassifier())]),
                   n_jobs=-1,
                   param_distributions={'clf__bootstrap': [True, False],
                                        'clf__max_depth': [10, 50, 100, None],
                                        'clf__max_features': ['auto', 'sqrt'],
                                        'clf__min_samples_leaf': [1, 2, 4],
                                        'clf__min_samples_split': [2, 5, 10],
                                        'clf__n_estimators': [800, 1400, 2000],
                                        'clf__random_state': [3],
                                        'vect__max_df': (0.5, 0.75, 1.0),
                                        'vect__ngram_range': (

In [ ]:
rf_best_model = RandomForestClassifier(bootstrap = False,
                                       max_depth = 50,
                                       max_features = 'auto',
                                       min_samples_leaf = 1,
                                       n_estimators = 1400,
                                       random_state=3)

In [ ]:
ac_score_list, p_score_list, r_score_list, f1_score_list = [], [], [], []
rf_best_model.fit(X_train, y_train)
y_pred = rf_best_model.predict(X_test)
ac_score_list.append(accuracy_score(y_test, y_pred))
p_score_list.append(precision_score(y_test, y_pred, average='macro'))
r_score_list.append(recall_score(y_test, y_pred, average='macro'))
f1_score_list.append(f1_score(y_test, y_pred, average='macro'))
model_comparison_df = pd.DataFrame([ac_score_list, p_score_list, r_score_list, f1_score_list]).T
model_comparison_df.columns = ['accuracy_score', 'precision_score', 'recall_score', 'f1_score']
model_comparison_df = model_comparison_df.sort_values(by='f1_score', ascending=False)

In [ ]:
model_comparison_df

In [ ]:
filename = './output/finalized_model.sav'
joblib.dump(model, filename)
 
# some time later...
 
# load the model from disk
loaded_model = joblib.load(filename)
result = loaded_model.score(X_test, Y_test)
print(result)